In [1]:
import pickle

In [2]:
rets = {}

In [3]:
path = "anymal_tanh_pd_ret"
with open("{}.pkl".format(path), "rb+") as file:
    data = pickle.load(file)

In [4]:
# resort
for seed, epi_data in data.items():
    for obs_index, obs_ret in epi_data.items():
        if obs_index == "seed":
            continue
        resort = sorted(obs_ret, key=lambda x: x["neuron"]["neuron_index"], reverse=True)
        resort = sorted(resort, key=lambda x: x["neuron"]["layer"], reverse=True)
        epi_data[obs_index] = resort


In [5]:
# average
averaged_ret = {}
for seed, epi_data in data.items():
    for obs, obs_ret in epi_data.items():
        if obs == "seed":
            continue
        if obs in averaged_ret:
            for k,each_neuron in enumerate(obs_ret):
                assert averaged_ret[obs][k]["layer"] == each_neuron["neuron"]["layer"] and averaged_ret[obs][k]["neuron_index"] == each_neuron["neuron"]["neuron_index"], "{} {}".format(averaged_ret[obs][k]["neuron_index"], each_neuron["neuron"]["neuron_index"])
                averaged_ret[obs][k]["freq_diff"] = (averaged_ret[obs][k]["freq_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["freq_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["base_phase_diff"] = (averaged_ret[obs][k]["base_phase_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["base_phase_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["seed_num"]+=1
        else:
            averaged_ret[obs] = []
            for each_neuron in obs_ret:
                averaged_ret[obs].append({"layer": each_neuron["neuron"]["layer"],
                                   "neuron_index": each_neuron["neuron"]["neuron_index"],
                                   "freq_diff": each_neuron["error"]["freq_diff"],
                                   "base_phase_diff": each_neuron["error"]["base_phase_diff"],
                                   "seed_num": 1})
for obs_dim, avg_ret in averaged_ret.items():
    averaged_ret[obs_dim] = sorted(avg_ret, key=lambda x:x["freq_diff"],reverse=False)

rets[path] = averaged_ret

In [6]:
for key, value in averaged_ret.items():
    remove = []
    for k,unit in enumerate(value):
        if unit["neuron_index"]>=128 and unit["layer"]==0:
            remove.append(unit)
        elif unit["neuron_index"]>=64 and unit["layer"]==1:
            remove.append(unit)
        elif unit["neuron_index"]>=32 and unit["layer"]==2:
            remove.append(unit)
    for k in remove:
        value.remove(k)

In [10]:
print(averaged_ret[2])

[{'layer': 0, 'neuron_index': 56, 'freq_diff': 99.1573022606131, 'base_phase_diff': 0.5861204325191005, 'seed_num': 10}, {'layer': 0, 'neuron_index': 70, 'freq_diff': 100.52125484102109, 'base_phase_diff': 0.9442395035066881, 'seed_num': 10}, {'layer': 2, 'neuron_index': 24, 'freq_diff': 101.61859443643648, 'base_phase_diff': 0.2735359581133737, 'seed_num': 10}, {'layer': 0, 'neuron_index': 102, 'freq_diff': 101.89216982447861, 'base_phase_diff': 0.7719810891693614, 'seed_num': 10}, {'layer': 0, 'neuron_index': 93, 'freq_diff': 102.02694943099148, 'base_phase_diff': 0.8219603237293363, 'seed_num': 10}, {'layer': 0, 'neuron_index': 55, 'freq_diff': 102.1312812272536, 'base_phase_diff': 0.5994161043971641, 'seed_num': 10}, {'layer': 0, 'neuron_index': 1, 'freq_diff': 103.24328759487166, 'base_phase_diff': 0.7190508885555402, 'seed_num': 10}, {'layer': 2, 'neuron_index': 22, 'freq_diff': 104.79616189306516, 'base_phase_diff': 0.5852794169111737, 'seed_num': 10}, {'layer': 0, 'neuron_index

In [ ]:

# 12-23  +1
# 24-35 +1 497 230 40
print(rets["cassie_0_z_pd_ret"][0])